# Actividad Final — Métodos Estadísticos Computacionales


**Temas:** Diseño de experimentos computacional · Manual de software · Protocolo de presentación final.  
Este notebook reproduce el flujo de trabajo del informe (**prueba de hipótesis, regresión lineal y logística**) y añade: **regularización**, **tuning**, **curva de aprendizaje**, **mini-dashboard JupyterDash** y **exportación de `app.py`**.


## 0) Dependencias

In [ ]:

# Si lo necesitas, instala dependencias desde el proyecto:
# %pip install -r ../requirements-win.txt


## 1) Cargar datos

In [ ]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
plt.rcParams['figure.figsize']=(6,4)
plt.rcParams['axes.grid']=True

df = pd.read_csv(Path('../data/students_synth.csv'))
df.head()


In [ ]:
df.describe(include='all')

## 2) EDA

In [ ]:

plt.hist(df['puntuacion'], bins=20, alpha=0.7); plt.title('Distribución de puntuación'); plt.show()

for m in ['A','B']:
    plt.hist(df.loc[df['metodo']==m,'puntuacion'], bins=20, alpha=0.6, label=f'Método {m}')
plt.title('Puntuación por método'); plt.legend(); plt.show()

plt.scatter(df['horas_estudio'], df['puntuacion'], alpha=0.5)
plt.title('Horas de estudio vs Puntuación'); plt.xlabel('horas_estudio'); plt.ylabel('puntuacion'); plt.show()


## 3) Prueba de hipótesis (t de Welch)

In [ ]:

import scipy.stats as stats
pA = df.loc[df['metodo']=='A','puntuacion']
pB = df.loc[df['metodo']=='B','puntuacion']

t_stat, p_value = stats.ttest_ind(pA, pB, equal_var=False)
print(f"Media A={pA.mean():.2f}  Media B={pB.mean():.2f}\nt={t_stat:.3f}  p={p_value:.4f}")
print("Conclusión:", "Rechazo H0 (difieren)" if p_value<0.05 else "No rechazo H0")


## 4) Regresión lineal y regularización

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error, r2_score

X_lin = df[['horas_estudio','horas_suenio','gpa_previo']]
y_lin = df['puntuacion']
Xtr, Xte, ytr, yte = train_test_split(X_lin, y_lin, test_size=0.3, random_state=42)

lin = LinearRegression().fit(Xtr, ytr)
yhat = lin.predict(Xte)
print("Coef:", dict(zip(X_lin.columns, np.round(lin.coef_,3))), "Intercepto:", round(lin.intercept_,3))
print("MSE:", round(mean_squared_error(yte,yhat),2), "R²:", round(r2_score(yte,yhat),3))

alphas = np.logspace(-3,3,21)
ridge = RidgeCV(alphas=alphas, cv=5).fit(Xtr, ytr)
lasso = LassoCV(cv=5, random_state=42, max_iter=5000).fit(Xtr, ytr)
print("Ridge α*=", ridge.alpha_, "R²(test)=", round(ridge.score(Xte,yte),3))
print("Lasso α*=", round(lasso.alpha_,4), "R²(test)=", round(lasso.score(Xte,yte),3))


In [ ]:

plt.scatter(yte, yhat, alpha=0.6); 
mn, mx = yte.min(), yte.max()
plt.plot([mn,mx],[mn,mx],'k--'); plt.title('Lineal — Real vs Predicho'); plt.xlabel('Real'); plt.ylabel('Predicho'); plt.show()


## 5) Regresión logística (clasificación)

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix

X_log = pd.concat([df[['horas_estudio','horas_suenio','gpa_previo']], pd.get_dummies(df['metodo'], drop_first=True)], axis=1)
y_log = df['aprueba']

Xtr_l, Xte_l, ytr_l, yte_l = train_test_split(X_log, y_log, test_size=0.3, random_state=42, stratify=y_log)
scaler = StandardScaler()
Xtr_l_sc = scaler.fit_transform(Xtr_l)
Xte_l_sc = scaler.transform(Xte_l)

logit = LogisticRegression(solver='liblinear', max_iter=1500, C=1.0).fit(Xtr_l_sc, ytr_l)
proba = logit.predict_proba(Xte_l_sc)[:,1]
yhat = (proba >= 0.5).astype(int)

print(dict(
    accuracy = round(accuracy_score(yte_l, yhat),3),
    precision = round(precision_score(yte_l, yhat, zero_division=0),3),
    recall = round(recall_score(yte_l, yhat),3),
    f1 = round(f1_score(yte_l, yhat),3),
    roc_auc = round(roc_auc_score(yte_l, proba),3),
    cm = confusion_matrix(yte_l, yhat)
))


In [ ]:

fpr,tpr,thr = roc_curve(yte_l, proba)
plt.plot(fpr,tpr,label=f"AUC={roc_auc_score(yte_l,proba):.3f}")
plt.plot([0,1],[0,1],'k--'); plt.legend(); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.title('ROC Logística'); plt.show()


### Tuning de hiperparámetros (C)

In [ ]:

from sklearn.model_selection import GridSearchCV
grid = {'C': np.logspace(-3,3,13)}
gcv = GridSearchCV(LogisticRegression(solver='liblinear', max_iter=2000), grid, cv=5, scoring='roc_auc', n_jobs=-1)
gcv.fit(Xtr_l_sc, ytr_l)
best = gcv.best_estimator_
print("Mejor C:", gcv.best_params_, "ROC_AUC(CV):", round(gcv.best_score_,3))
print("ROC_AUC(test):", round(roc_auc_score(yte_l, best.predict_proba(Xte_l_sc)[:,1]),3))


### Curva de aprendizaje (diseño experimental básico)

In [ ]:

from sklearn.model_selection import learning_curve
sizes, tr_sc, te_sc = learning_curve(LinearRegression(), X_lin, y_lin, cv=5, train_sizes=np.linspace(0.1,1.0,5), scoring='r2', n_jobs=-1)
plt.plot(sizes, tr_sc.mean(axis=1),'o-',label='Train R²')
plt.plot(sizes, te_sc.mean(axis=1),'o-',label='CV R²')
plt.legend(); plt.xlabel('Tamaño de entrenamiento'); plt.ylabel('R²'); plt.title('Curva de aprendizaje'); plt.show()


## 6) Mini-dashboard con JupyterDash (inline en notebook)

In [ ]:

from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output, dash_table
import plotly.express as px

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H3("Explorador — Estudiantes"),
    html.Div([
        html.Label("X"),
        dcc.Dropdown(id="x", options=[{"label":c,"value":c} for c in ["horas_estudio","horas_suenio","gpa_previo","indice_socioeco"]], value="horas_estudio"),
        html.Label("Y"),
        dcc.Dropdown(id="y", options=[{"label":"puntuacion","value":"puntuacion"}], value="puntuacion"),
        html.Label("Método"),
        dcc.Dropdown(id="m", options=[{"label":"Todos","value":"all"},{"label":"A","value":"A"},{"label":"B","value":"B"}], value="all"),
    ], style={"display":"flex","gap":"12px","flexWrap":"wrap"}),
    dcc.Graph(id="g"),
    dash_table.DataTable(id="tbl", data=df.head(50).to_dict("records"),
                         columns=[{"name":c,"id":c} for c in df.columns],
                         page_size=10, sort_action="native", filter_action="native", style_table={"overflowX":"auto"})
])

@app.callback(Output("g","figure"), Input("x","value"), Input("y","value"), Input("m","value"))
def _upd(x,y,m):
    dff = df.copy()
    if m != "all":
        dff = dff[dff["metodo"]==m]
    return px.scatter(dff, x=x, y=y, color="aprueba", trendline="ols")

print("Para ejecutarlo inline descomenta la siguiente línea:")
# app.run_server(mode='inline', port=8051, debug=False)


## 7) Exportar/actualizar `app.py` (para despliegue externo)

In [ ]:

from pathlib import Path
code = Path('../app.py').read_text(encoding='utf-8')
Path('../app.py').write_text(code, encoding='utf-8')
print("Se dejó listo ../app.py para desplegar en Render o HF Spaces.")


## 8) Conclusiones


- La prueba de hipótesis compara métodos (A/B) con la t de Welch.
- La regresión lineal explica la variación de la puntuación con estudio/sueño/GPA.
- La logística clasifica aprobación; ajustamos el umbral según objetivo.
- Regularización y *grid search* estabilizan modelos y controlan sobreajuste.
- El mini-dashboard permite narrativa y exploración interactiva; `app.py` habilita despliegue externo.
